<a href="https://colab.research.google.com/github/naimish75/Sentiment-Analysis-on-IMDB-Reviews/blob/main/IMDB_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing Sentiment Analysis On IMDB Reviews Using Bi-directional LSTMs

## Importing required libraries for the project

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional

from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gauravshetty/nltk_data...
2024-01-06 20:14:18.968836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gauravshetty/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gauravshetty/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Importing the dataset

Here we import the IMDB dataset. The dataset contains 2 columns. The first column is the review in the form of strings and the next is the sentiment which is our target variable. The reviews are divided into 2 sentiments 'Positive' and 'Negative'.

Dataset Link: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download

In [ ]:
df = pd. read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


## Pre-processing the data

Before giving the data as input to the neural network, we have to first pre-process the data so that it is compatible with our neural network model.

We start of by replacing our target variable with '1' and '0' for 'Positve' and 'Negative' respectively.

In [ ]:
df.sentiment.replace('positive',1,inplace=True)
df.sentiment.replace('negative',0,inplace=True)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [ ]:
df.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

Here we remove any HTML tags present in the string

In [ ]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

df.review = df.review.apply(clean)
df.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

We then remove any special characters and numeric data present in the string

In [ ]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalpha():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

df.review = df.review.apply(is_special)
df.review[0]

'One of the other reviewers has mentioned that after watching just   Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

Finally, we convert the string into lowercase characters

In [ ]:
def to_lower(text):
    return text.lower()

df.review = df.review.apply(to_lower)
df.review[0]

'one of the other reviewers has mentioned that after watching just   oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

Here we use WordNetLemmatizer from NLTK library to lemmatize all the words present in the dataset. We store this preprocessed data in a list which we are then going to use to create our embedded vectors

In [ ]:
wordnet = WordNetLemmatizer()
corpus = []

#print(df.iloc[1,0])

for i in range(0,len(df)):
 lem_review = [wordnet.lemmatize(word) for word in df.iloc[i,0].split() if word not in stopwords.words('english')]
 lem_review = ' '.join(lem_review)
 corpus.append(lem_review)

corpus[0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz m

## Creating one hot encoded vectors from preprocessed data

We start of by using the Corpus list that we created from the preprocessed data to create a one hot encoded vector.

In [ ]:
all_words = [word for words in corpus for word in words.split()]
voc_size = len(set(all_words))
print(voc_size)

onehot_repr = [one_hot(words, voc_size) for words in corpus]

92185


In [ ]:
# determining the maximum length of one hot encoded vectors

max_words = max(len(i) for i in onehot_repr)
max_words

1416

In [ ]:
# Padding and truncating to create same lenght vectors

sentence_len = 200
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_len)
#embedded_docs[0]

## Splitting the input dataset into training and testing

In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(np.array(df.sentiment.values))
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

## Setting the architecture of our neural network

### Model 1:

We start of by setting the size of embedded_vector as 120. We than add 5 layers of Bidirectional LSTM and then a dense layer. We set the activation function as 'Sigmoid' for our first model. And the loss as 'binary_crossentropy' and optimizer as 'adam'

In [ ]:
# creating the model architecture

embedded_vector = 120
model = Sequential()
model.add(Embedding(voc_size, embedded_vector, input_length=sentence_len))
model.add(Bidirectional(LSTM(200, return_sequences=True, input_shape=(120,1))))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 120)          11062200  
                                                                 
 bidirectional_5 (Bidirecti  (None, 100, 400)          513600    
 onal)                                                           
                                                                 
 bidirectional_6 (Bidirecti  (None, 100, 200)          400800    
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 100, 200)          240800    
 onal)                                                           
                                                                 
 bidirectional_8 (Bidirecti  (None, 100, 200)          240800    
 onal)                                                

 Using early stopping to prevent model from overfitting

In [ ]:
import tensorflow as tf

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='auto',
    min_delta=0,
    patience=3,
    verbose=0,
    restore_best_weights=True
)

Training the model

In [ ]:
import os

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    X_train,
    y_train,
    epochs=20,
    callbacks=[cp_callback, early_stopping],
    validation_data=(X_test,y_test),
    batch_size=64,
    verbose=1
)

Epoch 1/20
547/547 [==============================] - ETA: 0s - loss: 0.3713 - accuracy: 0.8356
Epoch 1: saving model to training_1/cp.ckpt
547/547 [==============================] - 982s 2s/step - loss: 0.3713 - accuracy: 0.8356 - val_loss: 0.3046 - val_accuracy: 0.8737
Epoch 2/20
547/547 [==============================] - ETA: 0s - loss: 0.1967 - accuracy: 0.9277
Epoch 2: saving model to training_1/cp.ckpt
547/547 [==============================] - 812s 1s/step - loss: 0.1967 - accuracy: 0.9277 - val_loss: 0.3292 - val_accuracy: 0.8719
Epoch 3/20
547/547 [==============================] - ETA: 0s - loss: 0.1152 - accuracy: 0.9602
Epoch 3: saving model to training_1/cp.ckpt
547/547 [==============================] - 489s 893ms/step - loss: 0.1152 - accuracy: 0.9602 - val_loss: 0.4032 - val_accuracy: 0.8645
Epoch 4/20
547/547 [==============================] - ETA: 0s - loss: 0.0664 - accuracy: 0.9787
Epoch 4: saving model to training_1/cp.ckpt
547/547 [==============================] 

Results of the testing:

Model accuracy: 87.37%

In [ ]:
result = model.evaluate(X_test, y_test)
print(result)

469/469 [==============================] - 70s 149ms/step - loss: 0.3046 - accuracy: 0.8737
[0.30457445979118347, 0.8737333416938782]


### Model 2:

Here we select a larger embedded_vector of size 200. We reduce the LSTM layers to 3 and change the activation function to 'tanh'

In [ ]:
# creating the model architecture for model 2

embedded_vector = 200
model_2 = Sequential()
model_2.add(Embedding(voc_size, embedded_vector, input_length=sentence_len))
model_2.add(Bidirectional(LSTM(300, return_sequences=True, input_shape=(200,1))))
model_2.add(Bidirectional(LSTM(100, return_sequences=True)))
model_2.add(Bidirectional(LSTM(100, return_sequences=True)))
model_2.add(Bidirectional(LSTM(50)))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 200)          18437000  
                                                                 
 bidirectional_19 (Bidirect  (None, 200, 600)          1202400   
 ional)                                                          
                                                                 
 bidirectional_20 (Bidirect  (None, 200, 200)          560800    
 ional)                                                          
                                                                 
 bidirectional_21 (Bidirect  (None, 200, 200)          240800    
 ional)                                                          
                                                                 
 bidirectional_22 (Bidirect  (None, 100)               100400    
 ional)                                               

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=False
)

Training the model

In [ ]:
checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model_2.fit(
    X_train,
    y_train,
    epochs=20,
    callbacks=[cp_callback, early_stopping],
    validation_data=(X_test,y_test),
    batch_size=64,
    verbose=1
)

Epoch 1/20
547/547 [==============================] - ETA: 0s - loss: 0.3920 - accuracy: 0.8271
Epoch 1: saving model to training_3/cp.ckpt
547/547 [==============================] - 1385s 3s/step - loss: 0.3920 - accuracy: 0.8271 - val_loss: 0.3315 - val_accuracy: 0.8690
Epoch 2/20
547/547 [==============================] - ETA: 0s - loss: 0.2188 - accuracy: 0.9169
Epoch 2: saving model to training_3/cp.ckpt
547/547 [==============================] - 1282s 2s/step - loss: 0.2188 - accuracy: 0.9169 - val_loss: 0.3449 - val_accuracy: 0.8653
Epoch 3/20
547/547 [==============================] - ETA: 0s - loss: 0.1511 - accuracy: 0.9463
Epoch 3: saving model to training_3/cp.ckpt
547/547 [==============================] - 1258s 2s/step - loss: 0.1511 - accuracy: 0.9463 - val_loss: 0.3801 - val_accuracy: 0.8647
Epoch 4/20
547/547 [==============================] - ETA: 0s - loss: 0.1013 - accuracy: 0.9656
Epoch 4: saving model to training_3/cp.ckpt
547/547 [==============================] 

Testing the model.

In [ ]:
result = model_2.evaluate(X_test, y_test)
print(result)

469/469 [==============================] - 158s 337ms/step - loss: 0.3960 - accuracy: 0.8692
[0.3960320055484772, 0.8691999912261963]


## Future Changes

There is certainly room for changes in these models. A change in the architecture or a change in some paramter can improve the model. Further improvements to be done in this model with time.